In [4]:
import os
######## HF CACHE (LOAD BEFORE HF PACKAGES) ########
os.environ['HF_HOME'] = "./models"
print(f"Current huggingface cache dir: {os.environ['HF_HOME']}")

import math
import random
import numpy as np
import torch
from transformers import AutoModel, OPTForCausalLM, AutoTokenizer, LogitsProcessorList
from transformers import LlamaForCausalLM, LlamaTokenizer
from torch.cuda.amp import autocast
from watermark_processor import WatermarkLogitsProcessor, WatermarkDetector

hash_key = 15485863
random.seed(hash_key)
np.random.seed(hash_key)
torch.manual_seed(hash_key)
torch.cuda.manual_seed(hash_key)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
term_width = 80

Current huggingface cache dir: ./models


In [5]:
import json

ckpt_path = "ckpt/opt/init_0.25_1.75_default.pth"
prompt = "The Patently-O blog this week took a look at how the number of pending patent cases is changing. With the number of patent cases filed falling \u2013 Lex Machina this week noted a 33% drop in October compared to October 2013, for example \u2013 Patently-O dug into the pending cases to give another perspective on the state of patent litigation.\nIts findings are consistent with the conclusion that the number of patent litigations is declining, although it noted that the number of pending cases is still well above that in October 2010.\nThe Patently-O folks also had a look at how the USPTO is implementing Alice v CLS Bank in patent examination, in an article in the 2014 Patently-O Patent Law Journal. It ranked the top applicants with post-Alice allowance withdrawals. Unsurprisingly IBM, eBay and Microsoft were the top three assignees, named in 47, 19 and 15 applications respectively. Perhaps more surprisingly, gaming companies IGT and WMS Gaming are in the top 10. Also appearing are finance companies JP Morgan, American Express and Bank of America, which reflects the impact of Alice on financial services patents.\nThe Supreme Court has declined to hear the appeal of a dispute over the copyright on Sherlock Holmes. On November 2 it left intact a ruling that 50 works by Sir Arthur Conan Doyle featuring the famous detective are in the public domain.\nDoyle\u2019s estate had been trying to get writer Leslie Klinger to pay a licence for using the Sherlock Holmes character in a compendium of new stories. A fee was paid for the first The New Annotated Sherlock Holmes but Klinger refused to pay a licence fee for the second installment.\nIn June, the Seventh Circuit ruled that the copyright on 46 stories and four novels featuring Holmes were in the public domain. In August, Judge Richard Posner ordered Doyle\u2019s estate to pay legal fees to Klinger and branded its demands \u201ca form of extortion\u201d.\nThe past week threw up a bizarre copyright question: can you copyright a backside? As reported in a post on The IPKat blog, Kim Kardashian believes fitness model Jen Selter is copying her poses when posting pictures of her backside on Instagram. Kardashian has reportedly asked her lawyers to investigate whether there is a copyright infringement.\nThe IPKat blog gave a detailed analysis of the issues involved, from an EU perspective at least. It quickly establishes that you cannot copyright a body part because it is neither a work nor the author\u2019s own intellectual creation.The issue gets less clear when breast or bottom implants are involved, which could be seen as"

In [6]:
##### LOAD MODEL #####

device = "cuda" if torch.cuda.is_available() else "cpu"
d_type = torch.float16

model = OPTForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype=d_type).to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b", padding_side="left")
model.eval()
model_simcse = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-base", torch_dtype=d_type).to(device)
model_simcse.eval()
model_ppl = OPTForCausalLM.from_pretrained("facebook/opt-2.7b", torch_dtype=d_type).to(device)
model_ppl.eval()

embed_matrix = OPTForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype=d_type).to(device).get_input_embeddings().weight        

for name, param in model.named_parameters():
    param.requires_grad = False
for name, param in model_simcse.named_parameters():
    param.requires_grad = False
for name, param in model_ppl.named_parameters():
    param.requires_grad = False

c:\Users\17189\.conda\envs\wm\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\17189\Documents\GitHub\TS_watermark\models\hub\models--facebook--opt-2.7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
##### GENERATION #####

watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                            ckpt_path=ckpt_path, 
                                            embed_matrix = embed_matrix)

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

attention_masks = (input_ids != tokenizer.pad_token_id).long()
prefix_len = input_ids.shape[1]

sample = dict(
    do_sample=True,
    top_k=50,
    top_p=1.0,
    temperature=1.0,
    attention_mask=attention_masks,
    min_new_tokens=200,
    max_new_tokens=200,
)

with torch.no_grad():
    with autocast():
        torch.manual_seed(hash_key)
        output_no_wm = model.generate(input_ids, 
                                **sample)
        
        torch.manual_seed(hash_key)
        output_w_wm = model.generate(input_ids, 
                                **sample,
                                logits_processor=LogitsProcessorList([watermark_processor]))

decoded_output_no_wm = tokenizer.batch_decode(output_no_wm[:,prefix_len:], skip_special_tokens=True)                    
decoded_output_w_wm = tokenizer.batch_decode(output_w_wm[:,prefix_len:], skip_special_tokens=True)                    

In [ ]:
decoded_output_no_wm, decoded_output_w_wm

([' a work even though breasts and bottoms make up less than 5% of the body. Also, Selter is merely a photographer’s model who does not write or create new ideas. She merely serves as a photo model. If Selter is the author, or if Kardashian is an author for her famous backside, then the court would have to decide who is a more successful creator.\nThe IPKat said that we might be heading for a legal tug-of-war. If Selter is the author and Kardashian is a famous photographer, then the issue will have to be decided by a court as to who is in a more positive light. The beauty influencer might think Selter’s models are taking a picture of Selter for a good cause where the photographer is not trying to copy the photos, but Selter would probably not think that’s the case.\nAlso in IPKat news, The IPKat Blog caught up with an IPKat and her'],
 [' a work even though breasts are not actual body parts.\nA few commentators have concluded that the situation is analogous to someone taking photos fro

In [ ]:
##### DETECTION #####

with torch.no_grad():
    watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                            device=device,
                                            tokenizer_opt=tokenizer,
                                            tokenizer_llama=None,
                                            embed_matrix=embed_matrix,
                                            ckpt_path=ckpt_path,
                                            normalizers="",
                                            ignore_repeated_bigrams=False)
    with autocast():
        score_dict = watermark_detector.detect(decoded_output_w_wm[0])
    z_score = score_dict['z_score'].item()
z_score

11.183034896850586

In [ ]:
##### PPL, SIMCSE #####

cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6) # function for SimCSE

with torch.no_grad():
    # PPL
    tokd_labels = output_w_wm.clone().detach()
    attention_masks = (output_w_wm != tokenizer.pad_token_id).long()
    tokd_labels[:,:prefix_len+1] = -100 
    log_ppl = model_ppl(output_w_wm, attention_mask=attention_masks, labels=tokd_labels).loss.item()

    # SimCSE: use the last 5 tokens of prompt and the generated text to compute.
    attention_masks = torch.ones_like(output_w_wm[:, (prefix_len-5):]) # should all be 1, since we set the new_token = 200
    embed_wm = model_simcse(output_w_wm[:, (prefix_len-5):], attention_mask=attention_masks, output_hidden_states=True, return_dict=True).pooler_output
    embed_no_wm = model_simcse(output_no_wm[:, (prefix_len-5):], attention_mask=attention_masks, output_hidden_states=True, return_dict=True).pooler_output
    simcse = cos(embed_wm[0], embed_no_wm[0]).item()  

math.exp(log_ppl), simcse

(12.692555303381242, 0.8056640625)